In [1]:
import streamlit as st
import pandas as pd
import numpy as np

In [5]:
import pandas as pd
import os

data = os.getcwd() + ("/app/coffee_may2023.csv")
#data = "coffee_may2023.csv" 
print(data)
df = pd.read_csv(data)
pd.set_option('display.max_columns', None)
df.columns


e:\Code\coffeerec/app/coffee_may2023.csv


Index(['Unnamed: 0', 'ID', 'Country of Origin', 'Farm Name', 'Lot Number',
       'Mill', 'ICO Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Number of Bags', 'Bag Weight', 'In-Country Partner', 'Harvest Year',
       'Grading Date', 'Owner', 'Variety', 'Status', 'Processing Method',
       'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
       'Uniformity', 'Clean Cup', 'Sweetness', 'Overall', 'Defects',
       'Total Cup Points', 'Moisture Percentage', 'Category One Defects',
       'Quakers', 'Color', 'Category Two Defects', 'Expiration',
       'Certification Body', 'Certification Address', 'Certification Contact'],
      dtype='object')

In [5]:
#from ydata_profiling import ProfileReport
from pandas_profiling import ProfileReport
#from streamlit_pandas_profiling import st_profile_report

pr = ProfileReport(df, minimal=True)
pr
#pr.to_file("app/report.json")
#pr.to_file("app/report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df[(df['Country of Origin'] == "Costa Rica")]


,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,1,22 kg,Japan Coffee Exchange,2022,"September 21st, 2022",Coffee Quality Union,Gesha,Completed,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
61,61,61,Costa Rica,Gamboa Farm,12075,Beneficio Montañas del Diamante,5-0025-0131,CECA S.A.,1850,Tarrazu,Martin Gutierrez,260,69 kg,Specialty Coffee Association of Costa Rica,2022 / 2023,"April 19th, 2023",Joselyn Morales Quesada,Caturra,Completed,Washed / Wet,7.58,7.83,7.50,7.67,7.83,7.67,10.0,10.0,10.0,8.58,0.0,84.67,10.0,0,1,green,2,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
68,68,68,Costa Rica,La Cumbre,CQU2022018,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,1,22 kg,Japan Coffee Exchange,2022,"October 4th, 2022",Coffee Quality Union,Gesha,Completed,Washed / Wet,7.75,7.83,7.67,7.83,7.75,7.75,10.0,10.0,10.0,7.92,0.0,84.50,11.7,0,1,green,0,"October 4th, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
93,93,93,Costa Rica,Gamboa Farm,12077-01,Beneficio Montañas del Diamante,5-0025-0132,CECA S.A.,1850,Tarrazu,Martin Gutierrez,250,35 kg,Specialty Coffee Association of Costa Rica,2022 / 2023,"April 19th, 2023",Joselyn Morales Quesada,Caturra,Completed,Washed / Wet,7.67,7.92,7.58,7.75,7.67,7.58,10.0,10.0,10.0,7.75,0.0,83.92,9.5,0,0,green,2,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
125,125,125,Costa Rica,Gamboa Farm,12074,Beneficio Montañas del Diamante,5-0025-0130,CECA S.A.,1850,Tarrazu,Martin Gutierrez,320,69 kg,Specialty Coffee Association of Costa Rica,2022 / 2023,"April 19th, 2023",Joselyn Morales Quesada,Caturra,Completed,Washed / Wet,7.58,7.75,7.42,7.67,7.50,7.75,10.0,10.0,10.0,7.67,0.0,83.33,0.0,0,0,green,3,"April 18th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
148,148,148,Costa Rica,AMA EL AROMA Micro lot - Sunrise,"The 2022 Pacific Rim Coffee Summit,CR01",Micro mill,NaN,Taiwan Coffee Laboratory,1450,west Villege,various producers,10,30 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,SL28,Completed,Pulped natural / honey,7.33,7.50,7.25,7.92,7.75,7.58,10.0,10.0,10.0,7.42,0.0,82.75,11.8,0,0,pale yellow,1,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
176,176,176,Costa Rica,San Diego Mill,S019194 - P04178,Dosmil,5-0015-0228,Beneficios Volcafé Costa Rica,1500,Corralillo Tarrazu,San Diego Mill,500,30 kg,Specialty Coffee Association of Costa Rica,2018 / 2019,"March 29th, 2023",Volcafe Japan,Caturra,Completed,Washed / Wet,7.42,7.42,7.42,7.33,7.58,7.33,10.0,10.0,10.0,7.50,0.0,82.00,10.5,1,2,green,8,"March 28th, 2024",Specialty Coffee Association of Costa Rica,"Blvd Rohrmoser, de Prisma Dental 200 norte y 2...",Noelia Villalobos - (506) 2220 0685
187,187,187,Costa Rica,HACIENDA SAN IGNACIO,OIC 5-019-0037/ Contenedor: DRYU3106597 / Clie...,Beneficio La Gior

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler


# Sample data: movies dataset with 'title' and 'description' columns
'''
data = pd.DataFrame({
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D'],
    'description': [
        'This is a thrilling action movie with explosions and car chases.',
        'A heartwarming comedy about friends on a road trip.',
        'A drama about love, loss, and redemption.',
        'An animated adventure with talking animals.'
    ]
})
'''

data=df

data['item_features'] = data['Country of Origin'] + ' ' + data['Variety'] + ' ' + data['Aroma']+ ' ' + data['Flavor']+' ' + data['Aftertaste']+' ' + data['Sweetness']

scaler = MinMaxScaler()
data['rating_normalized'] = scaler.fit_transform(data['rating'].values.reshape(-1, 1))

# Create a TF-IDF vectorizer to convert descriptions into feature vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['item_features'])

# Calculate the cosine similarity between item descriptions
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations based on movie title
def get_recommendations(title, cosine_similarities):
    idx = data.index[data['Company'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]  # Exclude the movie itself (most similar)
    movie_indices = [i[0] for i in sim_scores]
    return data['Farm Name'].iloc[movie_indices]

# Example: Get recommendations for 'Movie A'
recommendations = get_recommendations('CECA S.A.', cosine_sim)
print(recommendations)


TypeError: can only concatenate str (not "float") to str

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import linear_kernel

# Sample data: movies dataset with 'title', 'description', 'genres', 'actors', and 'rating' columns
data = pd.DataFrame({
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D'],
    'description': [
        'This is a thrilling action movie with explosions and car chases.',
        'A heartwarming comedy about friends on a road trip.',
        'A drama about love, loss, and redemption.',
        'An animated adventure with talking animals.'
    ],
    'genres': [
        'Action',
        'Comedy',
        'Drama',
        'Animation'
    ],
    'actors': [
        'Actor A, Actor B',
        'Actor C, Actor D',
        'Actor A, Actor E',
        'Actor F, Actor G'
    ],
    'rating': [4.2, 3.8, 4.5, 4.0]
})

# Concatenate multiple columns into a single feature column for item representation
data['item_features'] = data['description'] + ' ' + data['genres'] + ' ' + data['actors']

# Normalize 'rating' column to [0, 1] range for feature representation
scaler = MinMaxScaler()
data['rating_normalized'] = scaler.fit_transform(data['rating'].values.reshape(-1, 1))

# Create a TF-IDF vectorizer to convert item_features into feature vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['item_features'])

# Combine the TF-IDF matrix with the normalized 'rating' column
tfidf_matrix_with_rating = pd.concat([pd.DataFrame(tfidf_matrix.toarray()), data['rating_normalized']], axis=1)

# Calculate the cosine similarity between items
cosine_sim = linear_kernel(tfidf_matrix_with_rating, tfidf_matrix_with_rating)

# Function to get movie recommendations based on movie title
def get_recommendations(title, cosine_similarities, data):
    idx = data.index[data['title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]  # Exclude the movie itself (most similar)
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]

# Example: Get recommendations for 'Movie A'
recommendations = get_recommendations('Movie A', cosine_sim, data)
print(recommendations)


2    Movie C
3    Movie D
1    Movie B
Name: title, dtype: object


In [1]:
import pycaret

# loading sample dataset from pycaret dataset module
from pycaret.datasets import get_data
data = get_data('diabetes')

from pycaret.classification import *
s = setup(data, target = 'Class variable', session_id = 123)

from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

exp.setup(data, target = 'Class variable', session_id = 123)

ModuleNotFoundError: No module named 'pycaret'

In [11]:
best = compare_models()



Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [13]:
#plot_model(best, plot = 'confusion_matrix')
evaluate_model(best)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [14]:
holdout_pred = predict_model(best)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7576,0.8568,0.5309,0.7049,0.6056,0.4356,0.4447


In [15]:
holdout_pred.head()


,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable,prediction_label,prediction_score
552,6,114,88,0,0,27.799999,0.247,66,0,0,0.8036
438,1,97,70,15,0,18.200001,0.147,21,0,0,0.9649
149,2,90,70,17,0,27.299999,0.085,22,0,0,0.9394
373,2,105,58,40,94,34.900002,0.225,25,0,0,0.7999
36,11,138,76,0,0,33.200001,0.420,35,0,1,0.6392


In [16]:
new_data = data.copy()
new_data.drop('Class variable', axis=1, inplace=True)
new_data.head()


,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years)
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [17]:
predictions = predict_model(best, data = new_data)
predictions.head()


,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),prediction_label,prediction_score
0,6,148,72,35,0,33.599998,0.627,50,1,0.6940
1,1,85,66,29,0,26.600000,0.351,31,0,0.9419
2,8,183,64,0,0,23.299999,0.672,32,1,0.7976
3,1,89,66,23,94,28.100000,0.167,21,0,0.9454
4,0,137,40,35,168,43.099998,2.288,33,1,0.8393


In [8]:
from collections import defaultdict

from surprise import Dataset, SVD


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin("ml-100k")
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
        print(uid, [iid for (iid, _) in user_ratings])


196 ['318', '603', '483', '480', '114', '187', '192', '357', '963', '302']
186 ['427', '64', '603', '963', '483', '285', '498', '316', '496', '170']
22 ['64', '12', '114', '527', '357', '408', '302', '750', '97', '318']
244 ['408', '127', '12', '98', '178', '483', '589', '1142', '187', '134']
166 ['483', '50', '172', '98', '191', '174', '114', '169', '408', '496']
298 ['64', '313', '480', '190', '478', '12', '251', '272', '191', '173']
115 ['114', '408', '134', '169', '919', '428', '474', '238', '59', '272']
253 ['313', '603', '498', '169', '114', '357', '174', '480', '513', '316']
305 ['316', '611', '205', '185', '516', '589', '513', '136', '519', '656']
6 ['1142', '251', '705', '896', '654', '603', '179', '653', '963', '647']
62 ['408', '192', '484', '185', '661', '647', '1142', '169', '223', '513']
286 ['242', '98', '603', '515', '178', '318', '487', '607', '427', '64']
200 ['603', '197', '251', '136', '272', '963', '408', '659', '285', '185']
210 ['169', '408', '285', '100', '513',

In [9]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split


# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(df, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

AttributeError: 'DataFrame' object has no attribute 'raw_ratings'